# Learning from your previous model!

Hey guys! Welcome back to the transfer learning path.
In this tutorial you will learn how to use the previous train model to create a new one. Carful, this time will be a bit different than the last transfer learning tutorial.

### Progressive resizing

If you remember correctly, we've used the VGG-16 model to build our own model to predict if the image we provide as input is a cat or a dog.
We also initiate the input of the first layer in the VGG-16 model to accept image of size 64 by 64 shape.
This is all well and good but sadly we realize that such image shape reduce the precision for the image prediction, as the image quality is reduced.
The objective of this part is to increase progressively the shape of the input to train our model. This will teach you how to use your own model to apply transfer learning.

### Project overview

Lets import and check our previous model
with again all the necessary libraries.



In [1]:
from keras.models import Model, Sequential, load_model
from keras.layers import Dense,GlobalAveragePooling2D,Dropout,Flatten, Conv2D, MaxPooling2D

PREVIOUS_MODEL_PATH = './64_by_64.h5'



Using TensorFlow backend.


Lets observe again how our model looks like to clearly understand what we will need to change


In [2]:
previous_model = load_model(PREVIOUS_MODEL_PATH)
previous_model.summary()

W0913 12:28:07.296719 140735639835520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0913 12:28:07.318787 140735639835520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0913 12:28:07.347845 140735639835520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0913 12:28:07.562982 140735639835520 deprecation_wrapper.py:119] From /anaconda3/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0913 12:28:07.563727 140735639835520 deprecation_wrap

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 2, 2, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               524544    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 514       
Total params: 15,239,746
Trainable params: 525,058
Non-trainable params: 14,714,688
_________________________________________________________________


Lets check again how the VGG-16 Model in our layer 0

In [3]:
previous_model.layers[0].summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 128)       0         
__________

We can see that our input layer is of shape 64,64,3.
However, since we want to increase the details of the image taken for classification, we need to increase the number of pixels the input will take in. Lets double it this time. We will now have a input shape of 128, 128, 3.
To do so, we will need to change the input layer of the VGG-16, as well as the first Conv2D layer.

Why ?
Because the first conv2D layer is mapped to the input layer, meaning that it has a shape of 64 by 64.
As we are going to increase the number of imputed pixels to 128 by 128, our new conv2D will have to be of shape 128, 128 as well, while keeping the number of filter to 64.

This means that while we froze the other layers of the VGG-16, we will have to re-train the first conv2D to learn on the new input shape, then pass forward the information to the deeper layers who will remain frozen.

Why don't we have to change the other layers as well?  Because we will max pool our new conv2D layer of shape 128,128 into a shape 64, 64 which will match the shape of the second Conv2D layer in the VGG-16 model.



Now how are we going to proceed?

We will first initiate a new model with two layers, one conv2D for input shape 128,128
then a maxpooling layer which will divide the shape of the inputs by 2 (or more precisely, the shape of the filter image by 2) as 128 / 2 = 64.

In [4]:
model = Sequential()
model.add(Conv2D(64,kernel_size=(3,3),input_shape=(128,128,3),activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))


From here we just have to add the layers from the VGG-16 to our newly created model.
How so ? Since we have already frozen the VGG-16 in our previous model, we just need to loop over it and add each layers to our brand new model.

In [5]:
for layer in previous_model.layers[0].layers[2:]:
  # here we precise that we want to take all the layers from the second one to the last one in the VGG-16 model.
  model.add(layer)
  

Finally we add all the other layers in our previous model, while freezing them in the process

In [6]:
for layer in previous_model.layers[1:]:
  layer.trainable = False
  model.add(layer)
  
 



### training and testing our new model

time to train and test this new model and check its performance

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# defining the constants for the model training

BATCH_SIZE = 32
EPOCHS = 20
URL_TRAINING = './training_set' 
URL_TESTING = './test_set' 


# creating the image generator

generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_generator = ImageDataGenerator(
    rescale=1/255,
)


# creating the train and test sets

train_set = generator.flow_from_directory(URL_TRAINING, target_size=(128,128), batch_size=BATCH_SIZE)
test_set = test_generator.flow_from_directory(URL_TESTING, target_size=(128,128), batch_size=BATCH_SIZE)

 
# fitting the model

model.fit_generator(train_set, steps_per_epoch=len(train_set.filenames)//BATCH_SIZE, epochs=EPOCHS, validation_data = test_set, validation_steps=len(test_set.filenames)//BATCH_SIZE )



Now it is time to test it out

In [0]:
model.evaluate_generator(test_set, steps=BATCH_SIZE)

Awesome, you can see that our model is more accurate than the previous one!


Now it is your turn to create a new model from this one!

Try to double the input shape to 256,256 !

Good luck!